Combine wide, ground truth data (32 pixels wide) and the upsampled Halo data (21 pixels wide)

In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import xarray as xr

In [2]:
### Point to data
up_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing/upsampled')
up_files = list(up_dir.glob('3D*'))
up_files.sort()

gt_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing/wide_ground_truth')
gt_files = list(gt_dir.glob('gt*'))
gt_files.sort()

# assert len(up_files) == len(gt_files), "Mismatch in the number of processed campaigns!"
n_campaigns = len(up_files)

out_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing/wide_gt_and_up')
out_dir.mkdir(exist_ok=True)

In [4]:
jlo_upsampled = 6
jhi_upsampled = jlo_upsampled + 21
# nan_fill = 0.5 * (14.34118961191894 + 3.214014355133783)

for icampaign, files in enumerate(zip(up_files,gt_files)):
    if icampaign % 25 == 0: print(datetime.now(), icampaign, '...')
    up_file, gt_file = files[0], files[1]
    campaign_id = up_file.name[3:7]
    
    ds_up = xr.open_dataset(up_file)
    ds_gt = xr.open_dataset(gt_file)
    
    out_coords = {'y': ds_gt['y'],
                  'time': ds_gt['time']}
    ds_out = xr.Dataset(coords=out_coords)
    upsampled_data = np.nan * np.zeros_like(ds_gt['velocityx'].data)
    upsampled_data[:,jlo_upsampled:jhi_upsampled] = ds_up['U'].data.T
    
    ds_out['u_upsampled'] = (('time', 'y'), upsampled_data)
    ds_out['u_gt'] = (('time', 'y'), ds_gt['velocityx'].data)
    ds_out['v_gt'] = (('time', 'y'), ds_gt['velocityy'].data)
    ds_out['w_gt'] = (('time', 'y'), ds_gt['velocityz'].data)
    ds_out['T_gt'] = (('time', 'y'), ds_gt['temperature'].data)
    ds_out['TKE_gt'] = (('time', 'y'), ds_gt['tke'].data)
    
    ds_out.to_netcdf(Path(out_dir, f'gt_and_up{campaign_id}.nc'))

2024-06-12 09:34:03.133394 0 ...
2024-06-12 09:34:05.735959 25 ...
2024-06-12 09:34:08.317173 50 ...
2024-06-12 09:34:11.056241 75 ...
2024-06-12 09:34:13.718046 100 ...
2024-06-12 09:34:16.321033 125 ...
2024-06-12 09:34:19.019601 150 ...
2024-06-12 09:34:21.798377 175 ...
2024-06-12 09:34:24.630773 200 ...
2024-06-12 09:34:27.437077 225 ...
2024-06-12 09:34:29.973782 250 ...
2024-06-12 09:34:32.687850 275 ...
2024-06-12 09:34:35.507379 300 ...
2024-06-12 09:34:38.232809 325 ...
